<div class="usecase-title">ChildCare Facilities Analysis 2024</div>

<div class="usecase-authors"><b>Authored by: </b> Arjun Jamwal</div>

<div class="usecase-section-header">Scenario</div>

In a world full of varied places and attractions, getting about and seeing cities effectively might be difficult. The "Child Care Facilities Analysis" project makes use of Python's geographic analysis and mapping skills to provide a potent answer. This project combines multiple Python packages to give users a flexible platform. Get information from APIs with ease, opening up access to large datasets including addresses, landmarks, and places to hang out in a city.Users can freely customize their tour by filtering and exploring particular categories of interest, such as recreational regions or educational landmarks, with ease.Provide the best routes between user-specified destinations so that users can arrange their trips according to their favorite means of transportation—driving, walking, or cycling, for example.


This Python-based application enables users to navigate cities efficiently, find hidden treasures, and maximize their exploration experiences—whether they are parents foer there child, educators scouting educational locations, or hobbyists exploring cityscapes.

#### What is the use case? 

1. Easily navigate cities, finding monuments, eateries, and attractions along the way. Customize itineraries according to preferred forms of transportation.

2. Investigate educational locations such as museums, schools, and landmarks, planning the best routes for field trips or other educational activities involving students.

3. Analyze a property's closeness to public spaces, such as parks or schools, to help with well-informed real estate or urban development decisions.

4. Examine the distribution of facilities, flow of traffic, and infrastructure requirements for the development of cities and optimal accessibility.

5. Finding neaby best fit facility.


#### Walkthrough Steps

1. Featching Data from API.
2. Extracting relevent information & performing operations like removing null values and empty cells.
3. Visulization using folium
4. Filtereing Required Categories
5. Fexthing nearby Locations
6. Route Generation and Visualisation
7. Dynamic Data Display
8. Iterative Exploration


#### Brief Introduction about Dataset

This dataset offers a thorough overview of a city's heterogeneous topography, encompassing a variety of sites grouped under several themes and sub-themes. The name of the establishment ('feature_name'), the geographic coordinates ('latitude' and 'longitude'), and information about the topic and sub-theme classification of each entry are included to indicate a particular point of interest. This dataset includes a wide variety of categories, including retail stores, transportation hubs, educational institutions, leisure spaces, houses of worship, and more. This dataset is a useful resource for researchers, urban planners, and tourists alike because of its organized classification and precise location data, which enable geographical analysis, route optimization, tourism exploration, and a deeper comprehension of the city's complex landscape.


##### 1. Importing required Libraries 

In [1]:
# Import required modules
import requests
import pandas as pd
import folium          #used for geo-plotting
from geopy.distance import distance
from math import radians, sin, cos, sqrt, atan2
import requests

##### 2. Fetching Dataset from API 


The code that is given coordinates the retrieval of data from an API that is specific to Melbourne and probably contains information on theaters, schools, sports arenas, hospitals, and landmarks. It builds and searches the API using filters like limit and offset using a custom get_data function, allowing for batch data retrieval. A methodical loop gathers records one by one until the dataset is fully obtained, carefully compiling the information and adding it to an expanding list. This data eventually becomes a Pandas DataFrame, which offers a structured format for additional processing or analysis. Ensuring thorough data collecting through a methodical approach is contingent upon constant pagination inside the API.


In [2]:
# Function to get data from website using API
def get_data(base, data_url, offset=0):    
    # Set the filters, limit retrieves 20 rows at a time, offset says where to start data collection
    filters = f'records?limit={50}&offset={offset}&timezone=UTC'
    # Make the url from base, data url and filters variables stored ouside loop
    url = f'{base}{data_url}/{filters}'
    # Use the requests function to get the data
    result = requests.get(url)
    # Check that the request works, error code 200 = successful
    if result.status_code == 200:
        # Save results as a json file
        result_json = result.json()
        # Store a variable of max_results with total of dataset
        max_results = result_json['total_count']
        # Save the results key data to a list variable
        records = result_json['results']
    else:
        # If data is not collected correctly return the error
        print("ERROR GETTING DATA: ", result.status_code)
        max_results = 0
        records = []
    # At end of function, return the json results in records, max_results count and offset
    return [records, max_results, offset]



# Collect data from API
# Set offset increment
# (needs to match offset in get data function)
OFFSET_INCREMENT = 50
# Base url (this should be the same for all datasets)
BASE_URL = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
# Set specific url 
# (change this variable for the dataset you are working with)
SPECIFIC_PATH = 'landmarks-and-places-of-interest-including-schools-theatres-health-services-spor'

# Call the get data function, passing in variables above, save to result
result = get_data(BASE_URL, SPECIFIC_PATH)
# Save the records data returned in the get_data function to records list variable
records = result[0]
# Save the dataset size data returned in the get_data function to max_results variable
max_results = result[1] 
# Increase the offset returned in the get_data function (result[2]) by the offset increment
offset = result[2] + OFFSET_INCREMENT
# Check the length of the data returned and compare it against the max_results variable
# If the length o fthe data is less than the max_results, run the while loop
while len(records) != max_results:
    # Call the get data function again, passing in url, specific path and new offset value
    data = get_data(BASE_URL, SPECIFIC_PATH, offset)
    # Add the data collected to the existing records list
    records += data[0]
    # Increase the offset by the offset increment
    offset += OFFSET_INCREMENT
# Convert the records list of dictionaries into a pandas dataframe 
df = pd.DataFrame(records)

##### 3. Checking Null Values

The null_vals = df.isnull() code snippet.The DataFrame df is efficiently examined by sum(), which then calculates the count of null or missing values in each of its columns. This short code aggregates the count of nulls in each column by applying sum() to each column after using the isnull() method to create a boolean mask recognizing null entries in the DataFrame. The summary of the missing data distribution within the DataFrame is finally shown by print(null_vals), which is crucial for evaluating the quality of the data and maybe handling missing values in further analysis or data cleaning procedures.


In [3]:
null_vals = df.isnull().sum()

print(null_vals)

theme           0
sub_theme       0
feature_name    0
co_ordinates    0
dtype: int64


##### 4. Making Different column for co-ordinates

The code snippet pulls matching values from a nested dictionary inside the "co_ordinates" column to add "latitude" and "longitude" columns to the DataFrame df. The 'lat' and 'lon' values are fetched using apply() with lambda functions, filling the additional columns for location-based data analysis and accessibility.


In [4]:

df["latitide"] = df['co_ordinates'].apply(lambda x: x['lat'])
df["longitude"] = df['co_ordinates'].apply(lambda x: x['lon'])

##### 5. Mapping all co-ordinates  


Through the extraction of equivalent values from a nested dictionary within the "co_ordinates" column, the code snippet adds "latitude" and "longitude" columns to the DataFrame df. 'lat' (latitude) and 'lon' (longitude) values are fetched using apply() with lambda functions, filling the additional columns for location-based data interpretation and analysis.


In [5]:
map = folium.Map(location = [df['latitide'].iloc[0], df['longitude'].iloc[0]], zoom_start = 5)

for index, row in df.iterrows():
    folium.Marker([row['latitide'], row['longitude']]).add_to(map)
    
map

In [6]:
df

,theme,sub_theme,feature_name,co_ordinates,latitide,longitude
0,Transport,Railway Station,Flemington Bridge Railway Station,"{'lon': 144.939277838304, 'lat': -37.788164588...",-37.788165,144.939278
1,Mixed Use,Retail/Office/Carpark,Council House 2 (CH2),"{'lon': 144.966638432727, 'lat': -37.814259143...",-37.814259,144.966638
2,Leisure/Recreation,Informal Outdoor Facility (Park/Garden/Reserve),Carlton Gardens South,"{'lon': 144.971266479841, 'lat': -37.806068457...",-37.806068,144.971266
3,Place of Worship,Church,Wesley Church,"{'lon': 144.968168215633, 'lat': -37.810157644...",-37.810158,144.968168
4,Place of Worship,Church,St Augustines Church,"{'lon': 144.954862000132, 'lat': -37.816974135...",-37.816974,144.954862
...,...,...,...,...,...,...
237,Education Centre,School - Primary and Secondary Education,Melbourne Girls Grammar School,"{'lon': 144.985089428348, 'lat': -37.831536451...",-37.831536,144.985089
238,Retail,Department Store,Myer,"{'lon': 144.963855087868, 'lat': -37.813591198...",-37.813591,144.963855
239,Retail,Department Store,David Jones,"{'lon': 144.964373486798, 'lat': -37.813312726...",-37.813313,144.964373
240,Health Services,Medical Services,Mercy Private Hospital,"{'lon': 144.984435746587, 'lat': -37.811896809...",-37.811897,144.984436


##### 6. Finding unique Categories
This code sample uses the unique() function to get unique values from the DataFrame df's'sub_theme' column. All unique values detected in the'sub_theme' column are created into an array (unique_cat). The following for loop iterates over these distinct values and outputs each value (i) separately, providing a brief list of distinct categories or sub-themes that are present in the DataFrame's'sub_theme' column.

In [7]:
unique_cat = df['sub_theme'].unique()
for i in unique_cat:
    print(i)

Railway Station
Retail/Office/Carpark
Informal Outdoor Facility (Park/Garden/Reserve)
Church
Private Hospital
Police Station
Private Sports Club/Facility
Major Sports & Recreation Facility
Art Gallery/Museum
Theatre Live
Office
Function/Conference/Exhibition Centre
Public Buildings
Retail/Office
Public Hospital
Film & RV Studio
Current Construction Site
Observation Tower/Wheel
Further Education
Indoor Recreation Facility
Dwelling (House)
Store Yard
Tertiary (University)
Retail/Office/Residential/Carpark
Gymnasium/Health Club
Retail/Residential
Visitor Centre
Synagogue
Transport Terminal
Primary Schools
Secondary Schools
Government Building
Vacant Land - Undeveloped Site
Marina
Hostel
Outdoor Recreation Facility (Zoo, Golf Course)
Fire Station
Library
Cemetery
Retail
School - Primary and Secondary Education
Casino
Current Construction Site - Commercial
Cinema
Industrial (Manufacturing)
Aquarium
Bridge
Department Store
Medical Services


##### 7.  Making Dataset with required Categories

The code creates a new DataFrame called filter_loc and stores the rows from the DataFrame df that meet particular categories specified in the child_cat array.


In [8]:
child_cat = ["Informal Outdoor Facility (Park/Garden/Reserve)", 
             "Private Sports Club/Facility", 
             "Major Sports & Recreation Facility",
            "Indoor Recreation Facility",
            "Gymnasium/Health Club",
            "Primary Schools",
            "Secondary Schools",
            "Outdoor Recreation Facility (Zoo, Golf Course)",
            "School - Primary and Secondary Education",
            "Cinema",
            "Aquarium"]

filter_loc = df[df["sub_theme"].isin(child_cat)]



##### 8. Visualising Filtered Coordinates

The first point in the filtered DataFrame filter_loc serves as the center of a Folium map created by this code. Every location indicated by the "latitude" and "longitude" columns is represented by a marker on the map as iteratively going over each row of the filtered DataFrame. Ultimately, it generates a map that shows every marked place for the filtered categories inside the filter_loc DataFrame, providing a geographical representation of these particular categories or sites of interest.

In [9]:
map = folium.Map(location = [filter_loc['latitide'].iloc[0], filter_loc['longitude'].iloc[0]], zoom_start = 5)

for index, row in filter_loc.iterrows():
    folium.Marker([row['latitide'], row['longitude']]).add_to(map)
    
map

#### 9. Finding Nearby Locations according to user entered Co-ordinates

This code calculates distances between user-input coordinates and locations in a DataFrame. It filters locations within a specified radius of the input coordinates and displays them on a Folium map with markers indicating nearby points of interest.

In [10]:
#defining function to calculate distance between two coordinates
def cal_dist(lat1, lon1, lat2, lon2):
    rad_lat1 = radians(lat1)
    rad_lon1 = radians(lon1)
    rad_lat2 = radians(lat2)
    rad_lon2 = radians(lon2)
    
    lon_dist = rad_lon2 - rad_lon1
    lat_dist = rad_lat2 - rad_lat1
    
    cal_a = sin(lat_dist / 2)**2 + cos(rad_lat1) * cos(rad_lat2) * sin(lon_dist / 2) ** 2
    cal_c = 2 * atan2(sqrt(cal_a), sqrt( 1 - cal_a))
    worl_radius = 6371
    dist = worl_radius * cal_c
    return dist

#user entering co-ordinates

grab_lat = float(input("Enter the latitude:"))
grab_lon = float(input("Enter the longitude:"))
radi_find = 2

# code to find the nearby location by filtering coordinates

df["dist"] = df.apply(lambda row: cal_dist(grab_lat, grab_lon, row["latitide"], row["longitude"]), axis = 1)
radii_locs = df[df['dist'] <= radi_find]

#showing filtered locations
radis_map = folium.Map(location=[grab_lat, grab_lon], zoom_start = 12)

loc_list = []

for index,row in radii_locs.iterrows():
    text_hover = f"Location: {row['feature_name']}"
    folium.Marker([row["latitide"], row['longitude']], popup = text_hover).add_to(radis_map)
    loc_list.append((row['feature_name'], (row['latitide'], row['longitude'])))
radis_map


Enter the latitude:-37.7975831
Enter the longitude:144.9364779


##### 10 Finding route between two locations


Using the Open Source Routing Machine (OSRM) API, this code defines the method route_finder() to acquire route information between two specified geographic locations. Upon obtaining the user's location and a nearby destination, it proceeds to retrieve the route data from the OSRM API, taking into account the chosen mode of transportation (walking, cycling, or driving).

The code then centers the map on the user's location using Folium. It displays a polyline that shows the path between the user's current location and the destination of choice, depending on the travel mode selected, on the map. With the aid of this visual aid, users can see how to get from one place to another using the method of transportation of their choice.


In [11]:
import folium
import requests

def route_finder(user_loc, destiny, mode_travel):
    url = f"http://router.project-osrm.org/route/v1/{mode_travel}/{user_loc[1]},{user_loc[0]};{destiny[1]},{destiny[0]}?overview=full&geometries=geojson"
    response = requests.get(url)
    data_route = response.json()
    return data_route


#taking co-ordinates from useruser(user location)
user_lat = float(input("Enter your latitude: "))
user_lon = float(input("Enter your longitudes: "))
user_loc = (user_lat, user_lon)

#co-ordinates nearby to locations 

destiny_name = input("Enter the name of nearby location: ")
destiny = radii_locs[radii_locs['feature_name'] == destiny_name][['latitide', 'longitude']].iloc[0]
destiny = (destiny['latitide'], destiny['longitude'])





#selecting the means of travel
mode_travel = input("Select Travel Mode (walking/cycling/driving): ")


#generating the route between two points accoirding to travel mode
data_route = route_finder(user_loc, destiny, mode_travel)

#Displaying route using folium

mapper = folium.Map(location=[user_lat, user_lon], zoom_start = 12)
folium.Marker(user_loc, popup="Your Location").add_to(mapper)
folium.Marker(destiny, popup=destiny_name).add_to(mapper)

folium.PolyLine(locations=[(point[1], point[0]) for point in data_route['routes'][0]['geometry']['coordinates']],
               color = 'blue',
               weight = 3,
               opacity = 0.7).add_to(mapper)

mapper

Enter your latitude: -37.8323609
Enter your longitudes: 145.172095
Enter the name of nearby location: North Melbourne Uniting
Select Travel Mode (walking/cycling/driving): driving


In [12]:
df

,theme,sub_theme,feature_name,co_ordinates,latitide,longitude,dist
0,Transport,Railway Station,Flemington Bridge Railway Station,"{'lon': 144.939277838304, 'lat': -37.788164588...",-37.788165,144.939278,1.075801
1,Mixed Use,Retail/Office/Carpark,Council House 2 (CH2),"{'lon': 144.966638432727, 'lat': -37.814259143...",-37.814259,144.966638,3.234109
2,Leisure/Recreation,Informal Outdoor Facility (Park/Garden/Reserve),Carlton Gardens South,"{'lon': 144.971266479841, 'lat': -37.806068457...",-37.806068,144.971266,3.198810
3,Place of Worship,Church,Wesley Church,"{'lon': 144.968168215633, 'lat': -37.810157644...",-37.810158,144.968168,3.115578
4,Place of Worship,Church,St Augustines Church,"{'lon': 144.954862000132, 'lat': -37.816974135...",-37.816974,144.954862,2.694003
...,...,...,...,...,...,...,...
237,Education Centre,School - Primary and Secondary Education,Melbourne Girls Grammar School,"{'lon': 144.985089428348, 'lat': -37.831536451...",-37.831536,144.985089,5.699893
238,Retail,Department Store,Myer,"{'lon': 144.963855087868, 'lat': -37.813591198...",-37.813591,144.963855,2.992242
239,Retail,Department Store,David Jones,"{'lon': 144.964373486798, 'lat': -37.813312726...",-37.813313,144.964373,3.010883
240,Health Services,Medical Services,Mercy Private Hospital,"{'lon': 144.984435746587, 'lat': -37.811896809...",-37.811897,144.984436,4.503962
